In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
from dance_evaluation import *
import matplotlib.pyplot as plt

In [2]:
def calculate_metrics_with_oe(ref, calculated, tolerance=5):
    """
    Calculate Acc1, Acc2, Metric3, OE1, and OE2.
    Args:
        ref (np.ndarray): Reference BPMs.
        calculated (np.ndarray): Estimated BPMs.
        tolerance (float): Precision window (default 4%).
    Returns:
        dict: Dictionary with Acc1, Acc2, Metric3, OE1, and OE2 values.
    """
    # ref_tolerance = ref * tolerance
    ref_tolerance = tolerance
    
    # Acc1: Within 4% of reference BPM
    acc1_count = np.sum(np.abs(calculated - ref) <= ref_tolerance)
    
    # Acc2: Within 4% of reference BPM, double, or half
    acc2_count = np.sum(
        # (np.abs(calculated - ref) <= ref_tolerance) |
        (np.abs(calculated - 2 * ref) <= ref_tolerance) |
        (np.abs(calculated - ref / 2) <= ref_tolerance)
    )
    
    hits_idx = np.where(np.abs(calculated - ref) <= ref_tolerance)[0]
    hits_dbl_idx = np.where(np.abs(calculated - 2 * ref) <= ref_tolerance)[0]
    hits_hf_idx = np.where(np.abs(calculated - ref / 2) <= ref_tolerance)[0]
    
    error_half = np.abs(calculated - ref / 2)
    # error_1x = np.abs(calculated - ref)
    error_2x = np.abs(calculated - 2 * ref)
    
    scales = [1, 2, 0.5, 3, 1/3]

    # OE1: Overestimated BPM outside hierarchical relationships
    oe1_count = np.sum(
        (calculated > ref) &  # Overestimation
        ~np.any([np.abs(calculated - ref * scale) <= ref_tolerance for scale in scales], axis=0)  # Not within any scale
    )
    
    # OE2: Overestimated BPM within hierarchical relationships
    oe2_count = np.sum(
        (calculated > ref) &  # Overestimation
        np.any([np.abs(calculated - ref * scale) <= ref_tolerance for scale in scales], axis=0)  # Within any scale
    )
    
    total = len(ref)
    
    metrics = {
        "acc1": (acc1_count / total) * 100,
        "acc2": (acc2_count / total) * 100,
        "OE1": (oe1_count / total) * 100,
        "OE2": (oe2_count / total) * 100,
    }
    return metrics, hits_idx, hits_dbl_idx, hits_hf_idx

In [ ]:
calculated = np.array([10.1, 20.5, 30.8, 40.2, 50.0])
ref = np.array([10.0, 20.0, 30.0, 40.0, 50.0])
ref_tolerance = 0.5

# Condition
indices = np.where(np.abs(calculated - ref) <= ref_tolerance)[0]
indices

### Both Foot Onsets

In [3]:
base = "hand"
sname = "hand"

df_p1 = pd.read_csv(f"./{base}/max_xyz/{sname}_maxxyz_p1_50_150.csv")
df_p2 = pd.read_csv(f"./{base}/max_xyz/{sname}_maxxyz_p2_50_150.csv")
df_z1 = pd.read_csv(f"./{base}/max_xyz/{sname}_maxxyz_zero_bi_50_150.csv")
df_z2 = pd.read_csv(f"./{base}/max_xyz/{sname}_maxxyz_zero_uni_50_150.csv")


ref = df_p1["bpm"].to_numpy()
p1_tempoA = df_p1["tempoA"].to_numpy()
p1_tempoB = df_p1["tempoB"].to_numpy()
p1_tempoC = df_p1["tempoC"].to_numpy()

p2_tempoA = df_p2["tempoA"].to_numpy()
p2_tempoB = df_p2["tempoB"].to_numpy()
p2_tempoC = df_p2["tempoC"].to_numpy()

z1_tempoA = df_z1["tempoA"].to_numpy()
z1_tempoB = df_z1["tempoB"].to_numpy()
z1_tempoC = df_z1["tempoC"].to_numpy()

z2_tempoA = df_z2["tempoA"].to_numpy()
z2_tempoB = df_z2["tempoB"].to_numpy()
z2_tempoC = df_z2["tempoC"].to_numpy()

# Calculate metrics for Z-axis
# metricsA = calculate_metrics_with_oe(ref, tempoA, tolerance=8)
# metricsB = calculate_metrics_with_oe(ref, tempoB, tolerance=8)
# metricsC = calculate_metrics_with_oe(ref, tempoC, tolerance=8)

# metrics_list = [metricsA, metricsB, metricsC]

In [12]:
# Data for each experiment
experiments = {
    "zero_uni": [z1_tempoA, z1_tempoB, z1_tempoC],
    "zero_bi": [z2_tempoA, z2_tempoB, z2_tempoC],
    "peak_uni": [p1_tempoA, p1_tempoB, p1_tempoC],  # Replace ref for XYZ if not available
    "peak_bi": [p2_tempoA, p2_tempoB, p2_tempoC]   # Replace ref for XYZ if not available
}

method_labels = ["Method-1", "Method-2", "Method-3"]
results = {"experiment": [], "method": [], "acc1": [], "acc2": [], 
           "hits_idx": [], "hits_dbl_idx": [], "hits_hf_idx": []}

tolerance = 8
for exp_name, data in experiments.items():
    for axis_name, calculated in zip(method_labels, data):
        metrics, hits_idx, hits_dbl_idx, hits_hf_idx  = calculate_metrics_with_oe(ref, calculated, tolerance = tolerance)
        
        results["experiment"].append(exp_name)
        results["method"].append(axis_name)
        results["acc1"].append(metrics["acc1"])
        results["acc2"].append(metrics["acc2"])
        
        results["hits_idx"].append(hits_idx)
        results["hits_dbl_idx"].append(hits_dbl_idx)
        results["hits_hf_idx"].append(hits_hf_idx)

results_df = pd.DataFrame(results)

In [13]:

results_df

,experiment,method,acc1,acc2,hits_idx,hits_dbl_idx,hits_hf_idx
0,zero_uni,Method-1,0.000000,50.000000,[],[],"[0, 4, 5, 6, 7, 8, 9, 10, 12, 14, 15, 16, 20, ..."
1,zero_uni,Method-2,27.777778,0.000000,"[4, 5, 7, 8, 9, 12, 15, 21, 32, 34]",[],[]
2,zero_uni,Method-3,27.777778,0.000000,"[0, 5, 8, 9, 10, 12, 13, 15, 21, 25]",[],[]
3,zero_bi,Method-1,0.000000,50.000000,[],[],"[0, 1, 4, 6, 7, 8, 9, 12, 13, 15, 19, 20, 25, ..."
4,zero_bi,Method-2,36.111111,2.777778,"[0, 1, 4, 7, 8, 9, 12, 20, 28, 30, 31, 32, 33]",[],[35]
5,zero_bi,Method-3,33.333333,2.777778,"[0, 1, 4, 7, 9, 20, 24, 25, 30, 31, 32, 33]",[],[35]
6,peak_uni,Method-1,0.000000,0.000000,[],[],[]
7,peak_uni,Method-2,0.000000,66.666667,[],[],"[0, 2, 3, 4, 5, 6, 7, 8, 12, 13, 15, 18, 19, 2..."
8,peak_uni,Method-3,0.000000,91.666667,[],[],"[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15..."
9,peak_bi,Method-1,0.000000,11.111111,[],[],"[1, 11, 15, 31]"


In [ ]:

plt.figure(figsize=(8, 5))  # Adjust figure size for better readability
sns.barplot(data=results_df, x="experiment", y="acc2", palette="muted", hue="method")


plt.xlabel("Experiment")
plt.ylabel("Acc2 (%)")
plt.title(f"Selective x,y,z : Accuracy within {tolerance} bpm of double/half")
plt.xticks(rotation=45)  # Rotate labels if necessary
# plt.ylim(0, 100)  # Set consistent y-axis limits
plt.grid(axis="y", linestyle="--", alpha=0.7)


plt.tight_layout()
plt.savefig(f'./{base}/plots/{sname}_maxxyz_acc2.png')
plt.show()

In [ ]:
plt.figure(figsize=(6, 4))
plt.hist(tempoA/ref, bins=40)

plt.xlabel("Ratio")
plt.ylabel("Number of instance")
plt.title(f"Histograms of ratio between estimated tempo and correct tempo")
plt.show()